In [ ]:
!gdown --id 1zoJp9Z19qRflaTiZ3Osnj6EXI_9YK_O1


#1-5iThilGeWEc00S3WOyGgupYwJS0EsbN        data without any thing hhhhhhhhh

#https://drive.google.com/file/d/1zoJp9Z19qRflaTiZ3Osnj6EXI_9YK_O1/view?usp=sharing   Aug_data & preprocessing

#https://drive.google.com/file/d/1U5WvGLhki5Tjot8O7hGCoWOzJQiEt6oa/view?usp=sharing      TTV2_Aug_data

#https://drive.google.com/file/d/1QaSpkDcwcV4wfEUwDOdbcr-BpDgQGEJu/view?usp=sharing      TTV_Aug_data

#https://drive.google.com/file/d/1lgLRU4jGo9PryAJ7hruVMO5FgaLIh9Hr/view?usp=sharing      1lgLRU4jGo9PryAJ7hruVMO5FgaLIh9Hr      Aug_data

In [ ]:
#unzip folder
#!unzip All_resized_data.zip

!unrar x Preprocessed_Dataset.rar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Image, display
import numpy as np
import argparse
import cv2
import os

import shutil
import pandas as pd
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import glob

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import warnings
warnings.filterwarnings('ignore')

# Data science tools
import numpy as np
import pandas as pd
import os

# Image manipulations
from PIL import Image
# Useful for examining network
from torchsummary import summary
# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 14


In [ ]:
base_dir = '/content/Preprocessed_Dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


# Directory with our training cat pictures
train_melanoma_dir = os.path.join(train_dir, 'melanoma')

# Directory with our training dog pictures
train_not_melanoma_dir = os.path.join(train_dir, 'not_melanoma')

# Directory with our validation cat pictures
validation_melanoma_dir = os.path.join(validation_dir, 'melanoma')

# Directory with our validation dog pictures
validation_not_melanoma_dir = os.path.join(validation_dir, 'not_melanoma')

# Directory with our validation cat pictures
test_melanoma_dir = os.path.join(test_dir, 'melanoma')

# Directory with our validation dog pictures
test_not_melanoma_dir = os.path.join(test_dir, 'not_melanoma')

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Note that the test data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))


# Flow validation images in batches of 20 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(validation_dir,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(test_dir,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

In [ ]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [ ]:
#  Recall()
#  Precision()
presc = tf.keras.metrics.Precision(name='precision')
recall = tf.keras.metrics.Recall(name='recall')

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from keras.optimizers import Adam
optimizer = Adam(lr=0.001)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
 
base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(512, activation='relu'))
base_model.add(Dropout(0.3))
base_model.add(Dense(256, activation='relu'))
base_model.add(Dense(1, activation='sigmoid'))

In [ ]:
from datetime import datetime
import pytz

tz_NY = pytz.timezone('Africa/Cairo') 
current_time = datetime.now(tz_NY).strftime("%H-%M-%S")
file_name = 'ResNet50_finetune-'+current_time

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=10, verbose=1, mode='min', min_lr=5e-5)
checkpoint = ModelCheckpoint(file_name, monitor= 'val_loss', mode= 'min', save_best_only = True, verbose= 1)

callbacks=[lr_reduce,checkpoint]

In [ ]:
base_model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['acc', recall, presc])
#tf.keras.optimizers.SGD(lr=0.01)

In [ ]:
steps_per_epoch = 2585//20 #20 for batch_size

In [ ]:
resnet_history = base_model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = steps_per_epoch, epochs = 60, callbacks=callbacks)


In [ ]:
plt.figure(figsize=[8,6])
# x = 1000
plt.axis([0, 60, 0, 5])
plt.plot(resnet_history.history['loss'],'r',linewidth=3.0)

plt.plot(resnet_history.history['val_loss'],'b',linewidth=3.0)

plt.legend(['Training loss', 'Validation Loss'],fontsize=18)

plt.xlabel('Epochs ',fontsize=16)

plt.ylabel('Loss',fontsize=16)

plt.title('Loss Curves',fontsize=16)


In [ ]:
plt.figure(figsize=[8,6])

plt.plot(resnet_history.history['acc'],'r',linewidth=3.0)

plt.plot(resnet_history.history['val_acc'],'b',linewidth=3.0)

plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)

plt.xlabel('Epochs ',fontsize=16)

plt.ylabel('Accuracy',fontsize=16)

plt.title('Accuracy Curves',fontsize=16)

In [ ]:
plt.figure(figsize=[8,6])

plt.plot(resnet_history.history['precision'],'r',linewidth=3.0)

plt.plot(resnet_history.history['val_precision'],'b',linewidth=3.0)

plt.legend(['Training Precision', 'Validation Precision'],fontsize=18)

plt.xlabel('Epochs ',fontsize=16)

plt.ylabel('Precision',fontsize=16)

plt.title('Precision Curves',fontsize=16)

In [ ]:
plt.figure(figsize=[8,6])

plt.plot(resnet_history.history['recall'],'r',linewidth=3.0)

plt.plot(resnet_history.history['val_recall'],'b',linewidth=3.0)

plt.legend(['Training Recall', 'Validation Recall'],fontsize=18)

plt.xlabel('Epochs ',fontsize=16)

plt.ylabel('Recall',fontsize=16)

plt.title('Recall Curves',fontsize=16)

In [ ]:
model_save_file = "TTV2_ResNet_10-6-DO-0-3-lr-001.hdf5"
base_model.save(model_save_file)

In [ ]:
!mv TTV2_ResNet_10-6-DO-0-3-lr-001.hdf5 drive/MyDrive/melenoma_drive_data/saving_models

In [ ]:
# file_name

In [ ]:
!mv ResNet50_finetune-17-00-14 drive/MyDrive/melenoma_drive_data/saving_models_folder

In [ ]:
Y_pred = base_model.predict_generator(test_generator)

In [ ]:
resnet_history.history

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/melenoma_drive_data/saving_models/TTV2_ResNet_10-1.hdf5')
